# Homeostasis à-la-SparseNet on the activation probability

In this notebook, we test the convergence of SparseNet as a function of different learning parameters. This shows the relative robusteness of this method according to the coding parameters, but also the importance of homeostasis to obtain an efficient set of filters.

See also :
* http://blog.invibe.net/posts/2015-05-05-reproducing-olshausens-classical-sparsenet.html for a description of how SparseNet is implemented in the scikit-learn package
* http://blog.invibe.net/posts/2015-05-06-reproducing-olshausens-classical-sparsenet-part-2.html for a descrtiption of how we managed to implement the homeostasis

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
from shl_scripts import SHL_set
tag = 'HAP'
experiments = SHL_set({})

In [4]:
%%writefile {experiments.shl.cache_dir}/{tag}.py

import numpy as np
np.set_printoptions(precision=2, suppress=True)

try:
    import sys
    command = sys.argv[1]
except:
    command = 'run'
try:
    n_jobs = sys.argv[2]
except:
    n_jobs = 1
    n_jobs = 4
    n_jobs = 9
    n_jobs = 10
    n_jobs = 35

from shl_scripts.shl_experiments import SHL_set
tag = 'HAP'
opts = dict(homeo_method=tag, verbose=0)


experiments = SHL_set(opts, tag=tag)

variables = ['eta']
variables = ['eta', 'alpha_homeo']
variables = ['eta', 'eta_homeo', 'alpha_MP', 'eta_precision', ]
variables = ['eta', 'eta_homeo', 'alpha_homeo', 'alpha_MP']
variables = ['eta', 'eta_homeo', 'alpha_homeo']
variables = ['eta', 'eta_homeo', 'alpha_homeo', 'l0_sparseness']

list_figures = []
list_figures = ['show_dico', ]
list_figures = ['show_dico', 'show_Pcum']

display_variables = ['error', 'logL', 'perror', 'aerror', 'MI', 'cputime', 'F']
display_variables = ['error', 'qerror', 'F']

if command == 'run':
    experiments.run(variables=variables, n_jobs=n_jobs)

if command == 'plot':
    import matplotlib.pyplot as plt
    for variable in variables:
        experiments = SHL_set(opts, tag=tag)
        experiments.scan(variable=variable, list_figures=list_figures, display='', fig_kwargs={'dim_graph':(2, 15)}, verbose=1)
        plt.show()

        for display_variable in display_variables:
            print('Time evolution of ', display_variable)
            fig, ax = experiments.scan(variable=variable, list_figures=[], display='dynamic', display_variable=display_variable)
            plt.show()
        plt.show()

        for display_variable in display_variables:
            print('Comparison at the end of learning for ', display_variable)
            fig, ax = experiments.scan(variable=variable, list_figures=[], display='final', 
                                       label=tag, display_variable=display_variable)
            plt.show()
        

Overwriting cache_dir/HAP.py


In [5]:
%run {experiments.shl.cache_dir}/{tag}.py load

In [6]:
!ls -l {experiments.shl.cache_dir}/{tag}*
!rm -fr {experiments.shl.cache_dir}/{tag}*lock*

-rw-r--r--  1 laurentperrinet  staff     1846 Sep 27 12:47 cache_dir/HAP.py
-rw-r--r--  1 laurentperrinet  staff  2617680 Sep 27 12:34 cache_dir/HAP_alpha_MP=0.21654_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2617680 Sep 27 12:34 cache_dir/HAP_alpha_MP=0.26218_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2617680 Sep 27 12:34 cache_dir/HAP_alpha_MP=0.31744_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2617680 Sep 27 12:34 cache_dir/HAP_alpha_MP=0.38434_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2617680 Sep 27 12:34 cache_dir/HAP_alpha_MP=0.46534_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2617680 Sep 27 12:34 cache_dir/HAP_alpha_MP=0.56342_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2617680 Sep 27 12:34 cache_dir/HAP_alpha_MP=0.68216_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2617680 Sep 27 12:34 cache_dir/HAP_alpha_MP=0.82593_dico.pkl
-rw-r--r--  1 laurentperrinet  staff  2617680 Sep 27 12:38 cache_dir/HAP_alpha_MP=1.00000_dico.pkl
-rw-r--r--  1 laurentperrinet  st

In [7]:
# this removes older computations and restarts computations
!rm -fr {experiments.shl.cache_dir}/{tag}*pkl
!ls -l {experiments.shl.cache_dir}/{tag}*

-rw-r--r--  1 laurentperrinet  staff  1846 Sep 27 12:47 cache_dir/HAP.py


In [ ]:
%run {experiments.shl.cache_dir}/{tag}.py run

[Parallel(n_jobs=35)]: Using backend LokyBackend with 35 concurrent workers.


In [ ]:
%run {experiments.shl.cache_dir}/{tag}.py plot

In [ ]:
%load_ext version_information
%version_information shl_scripts, numpy, shl_scripts, pandas, matplotlib